In [14]:
!cd ./fine_tuned_sentence-transformers-distilbert-base-nli-stsb-mean-tokens/ && tar czvf ../model.tar.gz *

1_Pooling/
1_Pooling/config.json
README.md
config.json
config_sentence_transformers.json
eval/
eval/similarity_evaluation_pqa-valucation_results.csv
model.safetensors
modules.json
sentence_bert_config.json
similarity_evaluation_pqa-test_results.csv
special_tokens_map.json
tokenizer.json
tokenizer_config.json
vocab.txt


In [15]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

inputs = sagemaker_session.upload_data(path='model.tar.gz')

inputs

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


's3://sagemaker-us-east-1-862591134361/data/model.tar.gz'

In [16]:
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role

class StringPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')


In [17]:
import time

pytorch_model = PyTorchModel(model_data = inputs, 
                             role=role, 
                             entry_point ='inference.py',
                             source_dir = './endpoint_code',
                             py_version = 'py39', 
                             framework_version = '1.13.1',
                             predictor_cls=StringPredictor)

predictor = pytorch_model.deploy(instance_type='ml.m5d.large', 
                                 initial_instance_count=1, 
                                 endpoint_name = f'semantic-search-model-{int(time.time())}')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
------!

content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [18]:
import json

original_payload = 'Does this work with xbox?'
features = predictor.predict(original_payload)
vector_data = json.loads(features)

vector_data

[-0.232393279671669,
 0.0853954553604126,
 0.1242949590086937,
 0.29358524084091187,
 0.1708947718143463,
 -0.42499369382858276,
 1.081270456314087,
 0.27930697798728943,
 0.4404171109199524,
 -0.10191454738378525,
 -0.27905088663101196,
 0.5325591564178467,
 -0.2867881655693054,
 -0.3377244472503662,
 0.31371772289276123,
 -1.0429658889770508,
 -1.0136609077453613,
 0.42573869228363037,
 -0.361650675535202,
 -0.30018284916877747,
 -0.35042861104011536,
 0.05950886011123657,
 -0.31912165880203247,
 0.9444386959075928,
 -0.33490028977394104,
 0.5933817028999329,
 1.192330241203308,
 -0.3927040100097656,
 -0.21266815066337585,
 0.8315938711166382,
 -0.1848481297492981,
 0.0368524044752121,
 -0.2995677888393402,
 0.7402496337890625,
 0.15518224239349365,
 0.11479165405035019,
 0.038575202226638794,
 1.5410442352294922,
 0.8590444922447205,
 -0.9881664514541626,
 0.16882826387882233,
 -0.2866569459438324,
 1.0322178602218628,
 0.2515105605125427,
 0.05658130347728729,
 0.2455119639635086,


In [ ]:
import boto3

sagemaker_client = boto3.client('sagemaker')

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName='semantic-search-model-1698971244')